In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
wine = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
wine.head()

In [ ]:
print(wine["quality"].value_counts())
wine["quality"].hist()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler


X = wine.drop("quality", axis = 1)
y = wine["quality"]

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(X_scaled,
                                                      y, 
                                                      test_size = 0.25, 
                                                      random_state = 19)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier(n_neighbors = 1) 
knn.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score


y_pred = knn.predict(X_valid)
# print(y_pred)

accuracy_score(y_valid, y_pred)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
knn = KNeighborsClassifier(n_neighbors = 5)
scores = cross_val_score(knn, X, y, cv = kf, scoring = 'accuracy')
mean_score = scores.mean()

print(scores)
print(mean_score)

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51)}
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=5) # или cv=kf
knn_grid.fit(X_train, y_train)

In [ ]:
print(knn_grid.best_params_)
print(knn_grid.best_score_)

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors = 47)
y_pred = best_knn.fit(X_train, y_train).predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
x_values = np.arange(1, 51)
y_values = pd.DataFrame(knn_grid.cv_results_)["mean_test_score"]

plt.plot(x_values, y_values)
plt.show()